In [1]:
pip install linkify-it-py -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import re
import time

from linkify_it import LinkifyIt
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
from tqdm import tqdm
import requests
import xmltodict

%reload_ext autoreload
%autoreload 2
sys.path.append('../scripts')
import pbmd_tools as pbmd

In [2]:
pbmd.read_tokens("../.env")
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")
PUBMED_TOKEN = os.environ.get("PUBMED_TOKEN")

In [3]:
linkify = (
    LinkifyIt()
    .set({"fuzzy_email": False}) 
)

In [4]:
PMIDs = []
queries = []
for year in range(2009, 2023):
    queries.append(f'(("http"[Title/Abstract])) AND (("{year}/01/01"[Date - Publication] : "{year}/12/31"[Date - Publication]))')
stats = pbmd.get_forges_stat(queries, PMIDs)
len(PMIDs)

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:46<00:00,  3.36s/it]


47465

In [ ]:
#links_stat = {}

for PMID in tqdm(PMIDs[22192:]):
    try:
        summary = pbmd.get_summary(PMID, PUBMED_TOKEN, "../data/log_files/status_all_links.txt")
    except:
        try:
            summary = pbmd.get_summary(PMID, PUBMED_TOKEN, "../data/log_files/status_all_links.txt")
        except:
            continue
            
    abstract = pbmd.get_abstract_from_summary(summary, "../data/log_files/status_all_links.txt")
    
    if abstract != None:
    
        if linkify.test(abstract):
            for match in linkify.match(abstract):
                link = match.url
                try:
                    key = link.split('/')[2]
                except:
                    print(link)
                    continue
                if key in links_stat:
                    links_stat[key] += 1
                else:
                    links_stat[key] = 1

 51%|█████████████████████████████████████                                   | 12988/25273 [3:05:33<2:35:33,  1.32it/s]

In [7]:
sum(links_stat.values())

25383

In [6]:
dict(sorted(links_stat.items(), key=lambda x: x[1], reverse=True))

{'www.clinicaltrials.gov': 1204,
 'dx.doi.org': 1084,
 'clinicaltrials.gov': 588,
 'www.elsevier.com': 575,
 'www.diagnosticpathology.diagnomx.eu': 567,
 'ClinicalTrials.gov': 304,
 'www.controlled-trials.com': 289,
 'www.ncbi.nlm.nih.gov': 271,
 'sourceforge.net': 254,
 'www.webcitation.org': 235,
 'www.biomedcentral.com': 201,
 'code.google.com': 200,
 'links.lww.com': 188,
 'github.com': 171,
 'www.ebi.ac.uk': 155,
 'cran.r-project.org': 154,
 'radiology.rsna.org': 146,
 'youtu.be': 133,
 'www.trialregister.nl': 117,
 'www.anzctr.org.au': 109,
 'www.bioconductor.org': 93,
 'nt.ars-grin.gov': 86,
 'Clinicaltrials.gov': 78,
 'www.cdc.gov': 66,
 'www.umin.ac.jp': 66,
 'proteomecentral.proteomexchange.org': 62,
 'www.ClinicalTrials.gov': 54,
 'www.crd.york.ac.uk': 53,
 'bioconductor.org': 50,
 'www.who.int': 48,
 'www.springer.com': 46,
 'crdd.osdd.net': 42,
 'brm.psychonomic-journals.org': 41,
 'www.chictr.org': 39,
 'onlinelibrary.wiley.com': 39,
 'radiographics.rsna.org': 36,
 'sites